<a href="https://colab.research.google.com/github/micah-shull/LLMs/blob/main/LLM_040_sequential_chain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##  Sequential Chains: Key Concepts

### **1. Runnable Composition**
- **Pipelines are modular**: The `|` operator allows chaining together `Runnable` components in a clean, readable way.
- **Flexibility**: Each step is self-contained, making it easy to swap out components (e.g., use a different LLM or a new prompt).

**Key Takeaway**: LangChain's `Runnable` paradigm emphasizes modularity and reusability, allowing you to build workflows that are easy to maintain and extend.

---

### **2. Input Handling with `RunnableMap` and `RunnablePassthrough`**
- **`RunnableMap` and `RunnablePassthrough`** handle structured inputs like `{"name", "purpose"}` without modification.
- These ensure that your inputs flow into the pipeline correctly, particularly when dealing with multiple variables.

**Key Takeaway**: LangChain enables seamless handling of structured inputs, making pipelines adaptable to real-world use cases where multiple data points (like user inputs) need to be processed.

---

### **3. Multi-Step Processing**
- The chain demonstrates a **multi-step workflow**:
  1. Generate a professional email using a prompt.
  2. Pass the email draft as input to a summarization prompt.
  3. Process both prompts using the LLM.
  4. Parse the final result into a clean string.
- Each step builds on the output of the previous one, illustrating LangChain's **data flow management**.

**Key Takeaway**: LangChain allows chaining multiple tasks (generation, summarization, etc.) into a single, logical flow.

---

### **4. Prompt Engineering**
- The **prompt templates** are essential for guiding the LLM's behavior:
  - The first prompt provides specific instructions for generating a professional email.
  - The second prompt ensures a concise summary of the email.
  
**Key Takeaway**: Thoughtful prompt design is critical for getting the desired output from LLMs. LangChain makes prompts reusable and parameterized for dynamic input.

---

### **5. Integration with Models**
- The code integrates a specific LLM (`ChatOpenAI`) seamlessly into the workflow.
- This demonstrates how LangChain acts as a bridge between prompts, structured inputs, and powerful LLMs.

**Key Takeaway**: LangChain abstracts the complexity of integrating models, focusing on functionality rather than model-specific details.

---

### **6. Output Parsing**
- The `StrOutputParser` ensures that the final result is a clean, usable string, even when intermediate steps might return complex or verbose outputs.

**Key Takeaway**: LangChain provides utilities to control and format the final output, which is crucial for downstream applications.

---

### **7. Real-World Applicability**
- The pipeline solves a practical problem: generating and summarizing professional emails.
- It highlights how LangChain can be applied to real-world use cases, combining LLMs with structured workflows.

**Key Takeaway**: LangChain simplifies building task-specific AI applications by abstracting common workflows into reusable components.




In [3]:
# !pip install langchain
# !pip install openai
# !pip install python-dotenv
# !pip install langchain-openai

In [4]:
import os
from dotenv import load_dotenv
import openai
import json
import langchain
from langchain_openai import ChatOpenAI
from langchain.chains import SequentialChain
from langchain import PromptTemplate, LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import AIMessage, HumanMessage, SystemMessage
# Load environment variables from .env file
load_dotenv('/content/API_KEYS.env')
api_key = os.getenv("OPENAI_API_KEY")
# Set the environment variable globally for libraries like LangChain
os.environ["OPENAI_API_KEY"] = api_key
# Print the API key to confirm it's loaded correctly
print("API Key loaded from .env:",os.environ["OPENAI_API_KEY"][0:30])

API Key loaded from .env: sk-proj-e1GUWruINPRnrozmiakkRM


### Employee Review

### **Key Concepts**

1. **SequentialChain**:
   - Manages multiple chains in a logical sequence where each chain's output feeds into the next as input.
   - Ensures intermediate outputs (`review_summary`, `weaknesses`) are preserved for inspection or downstream use.

2. **LLMChain**:
   - Combines a prompt template and an LLM to perform specific tasks.
   - Each chain has an input (like `review` or `weaknesses`) and a defined output (e.g., `review_summary`).

3. **Prompt Engineering**:
   - Thoughtfully designed prompts ensure each step focuses on a clear and actionable goal:
     - Step 1: Summarize performance.
     - Step 2: Extract weaknesses.
     - Step 3: Propose solutions.

4. **Reusable Outputs**:
   - By specifying `output_key` for each chain, intermediate results are accessible for further use.

5. **Scalability**:
   - This modular design allows for easy addition of new steps or integration with external tools.

---

### **Why This Workflow is Valuable**

- **Automates Performance Review Analysis**:
   - Reduces manual effort in summarizing reviews, identifying weaknesses, and designing improvement plans.

- **Customizable**:
   - Prompts can be adjusted to align with organizational goals or performance criteria.

- **Transparent**:
   - Intermediate outputs (`review_summary`, `weaknesses`) provide clear visibility into each step of the process.



In [22]:
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain_openai import ChatOpenAI

# Initialize the LLM (Language Learning Model) with the desired model and configuration
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)  # GPT-4 model with moderate creativity

# Step 1: Define the first prompt template to summarize the performance review
template1 = "Give a summary of this employee's performance review:\n{review}"
prompt1 = ChatPromptTemplate.from_template(template1)

# Step 1: Create the first chain to generate a performance review summary
chain_1 = LLMChain(
    llm=llm,  # Use the LLM to process the prompt
    prompt=prompt1,  # Prompt for summarizing the review
    output_key="review_summary"  # Key for storing the summary output
)

# Step 2: Define the second prompt template to extract key weaknesses
template2 = "Identify key employee weaknesses in this review summary:\n{review_summary}"
prompt2 = ChatPromptTemplate.from_template(template2)

# Step 2: Create the second chain to extract employee weaknesses
chain_2 = LLMChain(
    llm=llm,  # Use the same LLM for this task
    prompt=prompt2,  # Prompt to identify weaknesses
    output_key="weaknesses"  # Key for storing the weaknesses output
)

# Step 3: Define the third prompt template to create a personalized improvement plan
template3 = "Create a personalized plan to help address and fix these weaknesses:\n{weaknesses}"
prompt3 = ChatPromptTemplate.from_template(template3)

# Step 3: Create the third chain to generate a personalized improvement plan
chain_3 = LLMChain(
    llm=llm,  # Use the same LLM for consistency
    prompt=prompt3,  # Prompt for creating an improvement plan
    output_key="final_plan"  # Key for storing the final plan output
)

# Combine all the chains into a SequentialChain
seq_chain = SequentialChain(
    chains=[chain_1, chain_2, chain_3],  # Execute the chains sequentially
    input_variables=['review'],  # Input required for the first chain
    output_variables=['review_summary', 'weaknesses', 'final_plan'],  # Outputs from all chains
    verbose=True  # Enable verbose mode to see detailed logs of each step
)

# The SequentialChain:
# 1. Takes 'review' as input.
# 2. Generates 'review_summary' from Chain 1.
# 3. Uses 'review_summary' to generate 'weaknesses' in Chain 2.
# 4. Uses 'weaknesses' to create 'final_plan' in Chain 3.


In [24]:
employee_review = '''
Employee Information:
Name: Joe Schmo
Position: Software Engineer
Date of Review: July 14, 2023

Strengths:
Joe is a highly skilled software engineer with a deep understanding of programming languages, algorithms, and software development best practices. His technical expertise shines through in his ability to efficiently solve complex problems and deliver high-quality code.

One of Joe's greatest strengths is his collaborative nature. He actively engages with cross-functional teams, contributing valuable insights and seeking input from others. His open-mindedness and willingness to learn from colleagues make him a true team player.

Joe consistently demonstrates initiative and self-motivation. He takes the lead in seeking out new projects and challenges, and his proactive attitude has led to significant improvements in existing processes and systems. His dedication to self-improvement and growth is commendable.

Another notable strength is Joe's adaptability. He has shown great flexibility in handling changing project requirements and learning new technologies. This adaptability allows him to seamlessly transition between different projects and tasks, making him a valuable asset to the team.

Joe's problem-solving skills are exceptional. He approaches issues with a logical mindset and consistently finds effective solutions, often thinking outside the box. His ability to break down complex problems into manageable parts is key to his success in resolving issues efficiently.

Weaknesses:
While Joe possesses numerous strengths, there are a few areas where he could benefit from improvement. One such area is time management. Occasionally, Joe struggles with effectively managing his time, resulting in missed deadlines or the need for additional support to complete tasks on time. Developing better prioritization and time management techniques would greatly enhance his efficiency.

Another area for improvement is Joe's written communication skills. While he communicates well verbally, there have been instances where his written documentation lacked clarity, leading to confusion among team members. Focusing on enhancing his written communication abilities will help him effectively convey ideas and instructions.

Additionally, Joe tends to take on too many responsibilities and hesitates to delegate tasks to others. This can result in an excessive workload and potential burnout. Encouraging him to delegate tasks appropriately will not only alleviate his own workload but also foster a more balanced and productive team environment.
'''

In [25]:
results = seq_chain(employee_review)

<ipython-input-25-ba61d33de059>:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = seq_chain(employee_review)




> Entering new SequentialChain chain...

> Finished chain.


In [28]:
results.keys()

dict_keys(['review', 'review_summary', 'weaknesses', 'final_plan'])

In [30]:
print(results['review_summary'])

**Performance Review Summary for Joe Schmo**

**Employee Information:**
- **Name:** Joe Schmo
- **Position:** Software Engineer
- **Date of Review:** July 14, 2023

**Strengths:**
Joe Schmo is recognized as a highly skilled software engineer with strong proficiency in programming languages, algorithms, and best practices in software development. His technical capabilities allow him to effectively solve complex problems and produce high-quality code.

Key strengths include:
- **Collaboration:** Joe actively engages with cross-functional teams, sharing insights and valuing input from others, showcasing his team-oriented mindset.
- **Initiative and Self-Motivation:** He proactively seeks out new projects and challenges, leading to significant enhancements in processes and systems.
- **Adaptability:** Joe demonstrates flexibility in managing changing project requirements and quickly learns new technologies, making him a valuable team member.
- **Problem-Solving Skills:** He employs a logic

In [ ]:
print(results['final_plan'])

To address and fix these weaknesses, the following personalized plan can be implemented for Joe:

1. Time management:

- Set clear goals and priorities: Joe should start each day by identifying the most important tasks that need to be accomplished and prioritize them accordingly. This will help him stay focused and ensure that he completes critical tasks on time.
- Break tasks into smaller, manageable chunks: Large projects can be overwhelming and lead to procrastination. Joe should break them down into smaller, more manageable tasks, setting specific deadlines for each subtask. This will help him track progress and stay on schedule.
- Use time management tools: Joe can utilize various time management tools, such as calendars, task management apps, or project management software, to help him stay organized and prioritize tasks effectively.

2. Written communication skills:

- Seek feedback and guidance: Joe should actively seek feedback from his colleagues or supervisors on his written

### Client Feedback

In [32]:
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain_openai import ChatOpenAI

# Initialize the LLM (Language Learning Model) with the desired model and configuration
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)  # GPT-4 model with moderate creativity

# Step 1: Define the first prompt template to summarize client feedback
template1 = "Summarize the following client feedback into key points:\n{feedback}"
prompt1 = ChatPromptTemplate.from_template(template1)

# Step 1: Create the first chain to generate a feedback summary
chain_1 = LLMChain(
    llm=llm,  # Use the LLM to process the prompt
    prompt=prompt1,  # Prompt for summarizing the feedback
    output_key="feedback_summary"  # Key for storing the summary output
)

# Step 2: Define the second prompt template to identify client pain points
template2 = "Based on the feedback summary, identify the client's main pain points:\n{feedback_summary}"
prompt2 = ChatPromptTemplate.from_template(template2)

# Step 2: Create the second chain to extract client pain points
chain_2 = LLMChain(
    llm=llm,  # Use the same LLM for this task
    prompt=prompt2,  # Prompt to identify pain points
    output_key="pain_points"  # Key for storing the pain points output
)

# Step 3: Define the third prompt template to create actionable recommendations
template3 = (
    "Create actionable recommendations to address the following client pain points:\n{pain_points}"
)
prompt3 = ChatPromptTemplate.from_template(template3)

# Step 3: Create the third chain to generate actionable recommendations
chain_3 = LLMChain(
    llm=llm,  # Use the same LLM for consistency
    prompt=prompt3,  # Prompt for creating actionable recommendations
    output_key="recommendations"  # Key for storing the recommendations output
)

# Combine all the chains into a SequentialChain
seq_chain = SequentialChain(
    chains=[chain_1, chain_2, chain_3],  # Execute the chains sequentially
    input_variables=['feedback'],  # Input required for the first chain
    output_variables=['feedback_summary', 'pain_points', 'recommendations'],  # Outputs from all chains
    verbose=True  # Enable verbose mode to see detailed logs of each step
)

# Example Input: Client Feedback
client_feedback = """
The product is useful, but the onboarding process was very confusing.
I also found the customer support to be unresponsive during critical times.
Overall, it gets the job done, but I expected better communication and clearer instructions.
"""


In [34]:
output = seq_chain(client_feedback)



> Entering new SequentialChain chain...

> Finished chain.


In [35]:
# Print Results
print("Feedback Summary:\n", output['feedback_summary'])
print("\nClient Pain Points:\n", output['pain_points'])
print("\nActionable Recommendations:\n", output['recommendations'])

Feedback Summary:
 Key Points from Client Feedback:

1. Product is useful and effective.
2. Onboarding process was confusing.
3. Customer support was unresponsive during critical times.
4. Expected better communication and clearer instructions.

Client Pain Points:
 Based on the feedback summary, the client's main pain points are:

1. **Confusing Onboarding Process**: The client found the onboarding process to be unclear, which may hinder their ability to fully utilize the product effectively.

2. **Unresponsive Customer Support**: The client experienced issues with customer support being unresponsive during critical times, which likely exacerbated their confusion and frustration.

3. **Lack of Clear Communication and Instructions**: The client expected better communication and clearer instructions, indicating a need for more comprehensive guidance and support throughout their experience with the product. 

These pain points highlight areas where the client feels improvements are neces

### **Email Writing Function:**

This function demonstrates how to use **LangChain's RunnablePipeline** to create a multi-step workflow for generating and summarizing a professional email. It integrates an LLM (OpenAI GPT-40-mini) to:

1. **Generate a professional email draft** based on the recipient's name and the email's purpose.
2. **Summarize the email** into a concise one-sentence summary, which could serve as a subject line or a quick preview.

The workflow ensures both outputs—the full email and the summary—are returned for further use.

---

### **What This Function Does:**

1. **Input Handling**:
   - Accepts structured input (`name` and `purpose`) and processes it through a pipeline.

2. **Email Drafting**:
   - Uses a prompt to instruct the LLM to generate a formal email.

3. **Summarization**:
   - Processes the email draft through another prompt to produce a summary.
---

### **Key Lessons:**

1. **Modularity**:
   - The pipeline is built with reusable components (`RunnableMap`, `RunnablePassthrough`, `RunnableLambda`), making it easy to expand or adjust for other tasks.

2. **Chaining Outputs**:
   - Outputs from one step feed into the next, demonstrating how LangChain facilitates multi-step workflows.

3. **Content Extraction**:
   - Handling LLM outputs like `AIMessage` and converting them into plain strings is essential for compatibility in complex pipelines.

4. **Parallel Processing**:
   - `RunnableMap` enables splitting outputs (e.g., email draft and summary) for simultaneous or independent processing.

5. **Prompt Engineering**:
   - Thoughtful prompts guide the LLM's behavior, demonstrating how different tasks (drafting and summarization) can be performed in sequence.

---

### **Use Cases and Applications:**

- **Email Automation**:
   - Draft professional emails and generate subject lines or summaries for preview or categorization.

- **Multi-Step Workflows**:
   - Useful for any task requiring generation, refinement, or summarization of content.

- **Scalability**:
   - The modular structure supports integration with additional tools (e.g., email-sending APIs, CRMs).

This function showcases how LangChain empowers developers to build structured, multi-step workflows that leverage LLMs efficiently and flexibly.

In [18]:
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnableMap
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

# Step 1: Define the LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

# Step 2: Define prompt templates for each task
# Task 1: Create an email draft
email_prompt = PromptTemplate(
    input_variables=["name", "purpose"],
    template=(
        "Write a professional email addressed to {name} "
        "with the purpose of {purpose}. Use a formal tone."
    )
)

# Task 2: Summarize the email
summary_prompt = PromptTemplate(
    input_variables=["email_draft"],
    template=(
        "Summarize the following email in one sentence:\n\n{email_draft}"
    )
)

# Step 3: Define a lambda to extract `AIMessage` content
extract_content = RunnableLambda(lambda x: x.content if hasattr(x, 'content') else x)

# Step 4: Define the pipeline
sequential_chain = (
    {
        "name": RunnablePassthrough(),
        "purpose": RunnablePassthrough()
    }  # Pass inputs directly
    | email_prompt  # Generate the email prompt
    | llm  # Generate the email
    | extract_content  # Extract content from AIMessage
    | RunnableMap(  # Map outputs for both email and summary
        {
            "email_draft": RunnablePassthrough(),  # Pass through the email draft
            "summary": summary_prompt | llm | extract_content  # Summarize and extract content
        }
    )
)

# Step 5: Run the chain with input
inputs = {"name": "Dr. Smith", "purpose": "scheduling a follow-up meeting"}
output = sequential_chain.invoke(inputs)

# Print results
print("Generated Email Draft:\n", output["email_draft"])
print("\nGenerated Summary:\n", output["summary"])


Generated Email Draft:
 Subject: Request for Follow-up Meeting 

Dear Dr. Smith,

I trust you are in good health. I am writing to express my interest in scheduling a follow-up meeting with you in the coming weeks.

Our previous meeting was enlightening and allowed us to touch base on several crucial points of our ongoing project. However, I believe that there are additional items that require our attention and in-depth discussion. These items are integral to the successful completion of our project.

Given the importance of these matters, I propose that we convene a follow-up meeting at a time that best suits your schedule. I am flexible and can adjust my availability accordingly.

Should you agree, kindly let me know your preferred date and time so that I may organize the necessary documents and prepare for our meeting.

Thank you in advance for your consideration. I look forward to our continued collaboration.

Best Regards,

[Your Name] 

[Your Position]

[Your Contact Information]
